In [11]:
import torch
import torch.nn as nn
from kasim_model import Kasim_Model
from kasim_tokenizer import Tokenizer   

k_tokenizer=Tokenizer("tokenizer.json")
sentence = "the capital of united states and the capital of france"
tokens = k_tokenizer.encode(sentence)
tokens = torch.tensor(tokens).unsqueeze(0)  # Add batch dimension


k_model = Kasim_Model(vocab_size=len(k_tokenizer.vocab), embed_dim=4, context_length=32)
k_model


C:\Users\kasim\AppData\Local\Temp\ipykernel_1892\1443349770.py:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  tokens = torch.tensor(tokens).unsqueeze(0)  # Add batch dimension


Kasim_Model(
  (embedding): Embedding(64, 4)
  (pos_embed): Embedding(32, 4)
  (self_attention): Kasim_Multi_Head_Attention(
    (multi_head_attention): MultiheadAttention(
      (out_proj): NonDynamicallyQuantizableLinear(in_features=4, out_features=4, bias=True)
    )
    (projection): Linear(in_features=4, out_features=4, bias=True)
  )
  (layer_norm): KasimRMSNorm()
)

In [7]:
sentence_meanings_whith_attention

tensor([[[ 6.5979e-01, -5.7790e-01,  1.6746e-01, -6.6660e-01],
         [ 3.4240e-01, -2.7027e-01,  9.1047e-02, -4.8475e-01],
         [ 5.7518e-02, -3.1539e-02, -4.7680e-02,  1.5087e-02],
         [ 1.6371e-01, -1.3049e-01, -1.6156e-02, -7.7814e-02],
         [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00],
         [ 2.2928e-01, -4.0661e-01,  4.0655e-02, -4.7999e-02],
         [ 9.8098e-02, -7.7434e-02,  2.6085e-02, -1.3888e-01],
         [ 4.0027e-02, -4.1049e-02, -2.0742e-02, -7.4604e-03],
         [ 8.4715e-02, -1.1791e-01,  7.1326e-03, -3.9499e-02],
         [ 2.4668e-01, -1.3704e-01, -1.2420e-02, -2.9597e-01],
         [-3.3944e-02,  1.3298e-01, -9.6689e-02,  4.4016e-02],
         [ 7.4738e-02,  1.7528e-01, -1.0887e-01, -1.3570e-01],
         [ 3.3311e-02,  2.5878e-01, -2.3116e-01,  1.1625e-02],
         [ 1.6578e-01, -3.4468e-02, -1.0083e-01, -1.1692e-01],
         [ 1.5113e-01, -1.7906e-01, -3.0293e-02, -5.1137e-02],
         [ 8.0528e-02, -6.2598e-02, -3.7555e-03, -7.790

In [9]:
import torch
from torch import nn
from kasim_causal_attention import KasimCausalAttention

class Kasim_Multi_Head_Attention(nn.Module):
    def __init__(self, embed_dim, num_heads,output_dim,context_length,dropout_rate=0):
        super().__init__()
        self.heads = nn.ModuleList([KasimCausalAttention(embed_dim,output_dim,context_length,dropout_rate) for _ in range(num_heads)])
            
    def forward(self, x):
        attention_outs=[]
        for head in self.heads:
            attention_outs.append(head(x))

        return torch.cat(attention_outs, dim=1)
        

In [4]:
mullti_head_attention = Kasim_Multi_Head_Attention(embed_dim=4, num_heads=2,output_dim=4,context_length=32)
out=mullti_head_attention(torch.randn(1,10,4))
out

tensor([[[-0.1291,  1.4138,  0.2813, -0.7558],
         [-0.1502,  0.5889, -0.1072, -0.4103],
         [-0.0934,  0.3247, -0.2100, -0.1048],
         [-0.1181,  0.6670,  0.0152, -0.3340],
         [-0.0481,  0.0661, -0.1237,  0.1204],
         [-0.1252,  0.0092, -0.1250, -0.0446],
         [-0.0291,  0.0139, -0.1151, -0.0398],
         [-0.0588,  0.4638, -0.0203, -0.2089],
         [ 0.4359, -0.3630,  0.0090,  0.2094],
         [ 0.0030, -0.1059, -0.0582,  0.1329],
         [-1.1579, -1.4339,  1.6106, -0.9951],
         [-0.7675, -0.9823,  1.1520, -0.3146],
         [-0.6445, -0.6725,  0.9247, -0.1615],
         [-0.1802, -0.1286,  0.2798,  0.0718],
         [-0.1963, -0.0689,  0.2455, -0.0268],
         [ 0.1226,  0.2401, -0.1386,  0.3176],
         [ 0.1814,  0.1904, -0.2165,  0.2567],
         [ 0.3426,  0.2555, -0.4573,  0.1675],
         [ 0.2824,  0.2331, -0.4230,  0.0329],
         [-0.0652, -0.0405,  0.0716,  0.0215]]], grad_fn=<CatBackward0>)

In [6]:
from kasim_layer_norm import KasimLayerNorm
norm_layer = KasimLayerNorm(4)
norm_layer(out)

tensor([[[-0.4193,  1.5316,  0.0996, -1.2118],
         [-0.3527,  1.6448, -0.2364, -1.0556],
         [-0.3545,  1.6888, -0.9243, -0.4100],
         [-0.4705,  1.6327, -0.1134, -1.0488],
         [-0.5425,  0.6541, -1.3347,  1.2231],
         [-0.9442,  1.4141, -0.9410,  0.4712],
         [ 0.2897,  1.2133, -1.5616,  0.0586],
         [-0.4070,  1.6631, -0.2544, -1.0017],
         [ 1.2372, -1.4851, -0.2174,  0.4652],
         [ 0.1124, -1.1037, -0.5715,  1.5628],
         [-0.5418, -0.7671,  1.7178, -0.4090],
         [-0.6479, -0.9060,  1.6579, -0.1039],
         [-0.7827, -0.8260,  1.6443, -0.0357],
         [-1.0509, -0.7668,  1.4814,  0.3363],
         [-1.1469, -0.3556,  1.5966, -0.0942],
         [-0.0740,  0.6057, -1.5861,  1.0545],
         [ 0.4197,  0.4682, -1.7109,  0.8230],
         [ 0.8438,  0.5670, -1.6982,  0.2874],
         [ 0.9015,  0.7246, -1.6316,  0.0055],
         [-1.1597, -0.6985,  1.3977,  0.4606]]], grad_fn=<MulBackward0>)

In [7]:
import torch
import torch.nn as nn

# Diyelim ki elimizde 2 cümle (batch=2), her cümlede 4 token var (seq_len=4), embedding boyutu 3
x = torch.randn(2, 4, 3)  
print("Orijinal tensor:\n", x)

# LayerNorm tanımla (embedding boyutuna göre)
layer_norm = nn.LayerNorm(3)

# LayerNorm uygula
out = layer_norm(x)

print("\nLayerNorm sonrası tensor:\n", out)

# Ortalama ve varyans kontrolü
print("\nOrtalama:", out.mean(-1))   # Her token embeddinginin ortalaması ~0
print("Varyans:", out.var(-1))       # Her token embeddinginin varyansı ~1


Orijinal tensor:
 tensor([[[ 1.3390,  0.1829,  0.4039],
         [ 1.0681,  0.3448, -1.7740],
         [ 0.0495,  0.2825,  0.2107],
         [-0.4569,  0.6818, -1.2625]],

        [[-0.1972, -0.3156, -0.4682],
         [ 0.4632,  0.1103,  2.7721],
         [-0.7135, -1.2071, -1.0836],
         [-0.5954, -0.0914, -0.1274]]])

LayerNorm sonrası tensor:
 tensor([[[ 1.3911, -0.9161, -0.4750],
         [ 0.9855,  0.3857, -1.3712],
         [-1.3479,  1.0424,  0.3055],
         [-0.1392,  1.2884, -1.1492]],

        [[ 1.1699,  0.1023, -1.2722],
         [-0.5523, -0.8513,  1.4036],
         [ 1.3726, -0.9807, -0.3919],
         [-1.4112,  0.7841,  0.6271]]], grad_fn=<NativeLayerNormBackward0>)

Ortalama: tensor([[ 7.9473e-08, -3.9736e-08, -3.9736e-08,  0.0000e+00],
        [-7.9473e-08,  3.9736e-08, -1.0928e-07,  5.9605e-08]],
       grad_fn=<MeanBackward1>)
Varyans: tensor([[1.4999, 1.5000, 1.4984, 1.5000],
        [1.4988, 1.5000, 1.4997, 1.4997]], grad_fn=<VarBackward0>)


In [13]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForCausalLM

q_tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen3-0.6B")
q_model = AutoModelForCausalLM.from_pretrained("Qwen/Qwen3-0.6B")

tokenizer_config.json: 0.00B [00:00, ?B/s]

c:\Users\kasim\anaconda3\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\kasim\.cache\huggingface\hub\models--Qwen--Qwen3-0.6B. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/726 [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/1.50G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

In [14]:
q_model

Qwen3ForCausalLM(
  (model): Qwen3Model(
    (embed_tokens): Embedding(151936, 1024)
    (layers): ModuleList(
      (0-27): 28 x Qwen3DecoderLayer(
        (self_attn): Qwen3Attention(
          (q_proj): Linear(in_features=1024, out_features=2048, bias=False)
          (k_proj): Linear(in_features=1024, out_features=1024, bias=False)
          (v_proj): Linear(in_features=1024, out_features=1024, bias=False)
          (o_proj): Linear(in_features=2048, out_features=1024, bias=False)
          (q_norm): Qwen3RMSNorm((128,), eps=1e-06)
          (k_norm): Qwen3RMSNorm((128,), eps=1e-06)
        )
        (mlp): Qwen3MLP(
          (gate_proj): Linear(in_features=1024, out_features=3072, bias=False)
          (up_proj): Linear(in_features=1024, out_features=3072, bias=False)
          (down_proj): Linear(in_features=3072, out_features=1024, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): Qwen3RMSNorm((1024,), eps=1e-06)
        (post_attention_layernorm): Qwe

In [15]:
k_model

Kasim_Model(
  (embedding): Embedding(64, 4)
  (pos_embed): Embedding(32, 4)
  (self_attention): Kasim_Multi_Head_Attention(
    (multi_head_attention): MultiheadAttention(
      (out_proj): NonDynamicallyQuantizableLinear(in_features=4, out_features=4, bias=True)
    )
    (projection): Linear(in_features=4, out_features=4, bias=True)
  )
  (layer_norm): KasimRMSNorm()
)